In [49]:
import cv2
import numpy as np
import os
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [ ]:
# Callbacks 
 
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True),
    ModelCheckpoint(filepath='model.keras', save_best_only=True),
]

In [ ]:
# Charger les données d'entraienement 

def load_data():
    # Initialiser les listes pour les données et les labels
    train_data = []
    train_labels = []
    test_data = []
    test_labels = []

    # Données oliwer et damien

    # for photos in ['oliwer', 'damien']: 
    for photos in ['oliwer']:
        train = f'processed/{photos}/train/' # Images d'entraînement

        for file in os.listdir(train):
            if file.endswith('.jpg') or file.endswith('.png'):
                img = os.path.join(train, file) # Lire le chemin de l'image
                image = cv2.imread(img, cv2.IMREAD_GRAYSCALE) # Transformer l'image en gris
                train_data.append(image) # Ajouter l'image à la liste des données d'entraînement
                train_labels.append(photos) # Ajouter le label correspondant

        for file in os.listdir(f'processed/{photos}/test/'):
            if file.endswith('.jpg') or file.endswith('.png'):
                img = os.path.join(f'processed/{photos}/test/', file) # Lire le chemin de l'image de test
                image = cv2.imread(img, cv2.IMREAD_GRAYSCALE) # Transformer l'image en gris
                test_data.append(image) # Ajouter l'image à la liste des données de test
                test_labels.append(photos) # Ajouter le label correspondant

    # Données aléatoires

    random_images = []
    random_dir = './lfw-deepfunneled/lfw-deepfunneled'  # Dossier avec des images aléatoires
    
    for person in os.listdir(random_dir):
        person_path = os.path.join(random_dir, person) # Chemin du sous-dossier de la personne
        if os.path.isdir(person_path):
            for filename in os.listdir(person_path):
                img_path = os.path.join(person_path, filename) # Chemin complet de l'image
                if filename.endswith('.jpg') or filename.endswith('.png'):
                    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) # Transformer l'image en gris
                    image = cv2.resize(image, (128, 128)) # Redimensionner l'image à 128x128 pixels
                    random_images.append(image) # Ajouter l'image à la liste des images aléatoires

    # Sélectionner 80 images pour l'entraînement et 20 pour le test
    np.random.shuffle(random_images)
    train_data.extend(random_images[:80])
    train_labels.extend(['random'] * 80)
    test_data.extend(random_images[80:100])
    test_labels.extend(['random'] * 20)

    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)

In [ ]:
# Préparation des données 

train_data, train_labels, test_data, test_labels = load_data() # Charger les données

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)  # Encoder les labels d'entraînement
test_labels = label_encoder.transform(test_labels)        # Encoder les labels de test

train_labels = to_categorical(train_labels, num_classes=3)  # Convertir les labels en categorical
test_labels = to_categorical(test_labels, num_classes=3)

train_data = train_data.reshape(-1, 128, 128, 1)  # Reshape pour le modèle CNN
test_data = test_data.reshape(-1, 128, 128, 1)